# Facial Emotion Recognition with Keras

## Project Definition

**Project Link:** https://www.kaggle.com/datasets/fer2013

**Challenge:** Classify facial expressions into 8 emotion categories using deep learning.

**Data:** Images of people showing different emotions (angry, disgust, fear, happy, neutral, sad, surprise, contempt).

**ML Type:** Supervised Multiclass Classification

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import cv2

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

TensorFlow version: 2.18.0
Keras version: 3.8.0


## Data Loading and Initial Look

In [2]:
!pip install -q kagglehub

# 3. Set the Kaggle config directory
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

import kagglehub

# Download latest version
path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

print("Folders in images/:")
print(os.listdir(path))
total_files = sum([len(files) for _, _, files in os.walk(path)])
print(f" Total image files found: {total_files}")


Path to dataset files: /kaggle/input/fer2013
Folders in images/:
['test', 'train']
 Total image files found: 35887


In [3]:
import cv2
from tensorflow.keras.utils import to_categorical

# Set image size and data paths
img_size = (48, 48)
base_path = path  # from kagglehub
train_path = os.path.join(base_path, "train")
test_path = os.path.join(base_path, "test")

# Helper to load images and labels
def load_images_from_folder(folder_path):
    images = []
    labels = []
    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        if not os.path.isdir(label_path):
            continue
        for file in os.listdir(label_path):
            if file.lower().endswith((".jpg", ".png", ".jpeg")):
                img_path = os.path.join(label_path, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, img_size)
                img = img.astype('float32') / 255.0
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels)

# Load train and test sets
print(" Loading training data...")
X_train, y_train = load_images_from_folder(train_path)
print(f" Loaded {X_train.shape[0]} training images")

print(" Loading testing data...")
X_test, y_test = load_images_from_folder(test_path)
print(f" Loaded {X_test.shape[0]} testing images")

# Encode labels
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_test_enc = label_encoder.transform(y_test)

# Convert labels to categorical
y_train_cat = to_categorical(y_train_enc, num_classes=len(label_encoder.classes_))
y_test_cat = to_categorical(y_test_enc, num_classes=len(label_encoder.classes_))

# Reshape input images for CNN
X_train = X_train.reshape(-1, 48, 48, 1)
X_test = X_test.reshape(-1, 48, 48, 1)

# Label mapping
label_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("\n Label map:")
for name, code in label_map.items():
    print(f"{name} → {code}")


 Loading training data...
 Loaded 28709 training images
 Loading testing data...
 Loaded 7178 testing images

 Label map:
angry → 0
disgust → 1
fear → 2
happy → 3
neutral → 4
sad → 5
surprise → 6
